# Primer konvolucijske mreže (CNN) 
Z uporabo knjižnice Keras drastično poenostavimo osnovno testiranje različnih modelov. Omogoča nam enostavno implementacijo raznoraznih modelov in tudi kasnejše fine nastavitve obstoječega modela.

Ta primer želi klasificirati nabor podatkov [MNIST](http://yann.lecun.com/exdb/mnist/).

In [1]:
import keras
import numpy as np 

from keras.layers import Conv2D, MaxPooling2D, Dense, Reshape, Flatten, Activation, Dropout
from keras.callbacks import TensorBoard

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()


def one_hot(obj, n_classes):
    obj_len = len(obj)
    a = np.array(obj)
    b = np.zeros((obj_len, n_classes))
    b[np.arange(obj_len), a] = 1

    return b


Using TensorFlow backend.


In [2]:
"""
Definicija modela, vsak sloj posebej.
Sequential tip modela predstavlja model, 
ki se izvaja linearno po svojih slojih.
"""
def model_setup(n_conv_layers, n_dense_layers):
    model = keras.models.Sequential()
    ### Prvi del modela - KONVOLUCIJA
    # osnovno preoblikovanje nabora podatkov
    model.add(Reshape((28,28,1), input_shape=(28,28)))
    
    if n_conv_layers == 2:
        # prvi konvolucijski sloj s 32 jedri
        model.add(Conv2D(32, kernel_size=5, strides=1, padding='same', activation='relu'))
        # prvo maksimalno združevanje
        model.add(MaxPooling2D(pool_size=2, strides=2))
        # drugi konvolucijski sloj s 64 jedri
        model.add(Conv2D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
        # drugo maksimalno združevanje
        model.add(MaxPooling2D(pool_size=2, strides=2))
        
    else:
        # prvi konvolucijski sloj s 48 jedri
        model.add(Conv2D(48, kernel_size=5, strides=1, padding='same', activation='relu'))
        # maksimalno združevanje
        model.add(MaxPooling2D(pool_size=2, strides=2))
    
    # preoblikovanje matrike iz 4-D v 2-D
    model.add(Flatten())

    ### Drugi del modela - KLASIFIKATOR
    if n_dense_layers == 2: 
        # prvi polno-povezan sloj s relu aktivacijo
        model.add(Dense(units=512, activation='relu'))
        # osipni sloj p = 0.5
        model.add(Dropout(0.5))
        # drugi polno-povezan sloj s relu aktivacijo
        model.add(Dense(units=512, activation='relu'))
        # osipni sloj p = 0.5
        model.add(Dropout(0.5))
    
    else:
        # osnovni (vmesni) polno-povezan sloj s relu aktivacijo
        model.add(Dense(units=512, activation='relu'))
        # osipni sloj p = 0.5
        model.add(Dropout(0.5))
        
    
    # zadnji polno-povezan sloj s softmax aktivacijo 
    model.add(Dense(units=10, activation='softmax'))
    
    return model
    

In [3]:
Optimizers = ['Adam']#, 'Adadelta', 'sgd', 'RMSprop']
n_dense_layers = [2, 1]
n_conv_layers = [2, 1]

# Pretvorba razrednih vektorjev
y_train = one_hot(y_train, 10)
y_test = one_hot(y_test, 10)

for opti in Optimizers:
    for n_dense in n_dense_layers:
        for n_conv in n_conv_layers:
            hparam = '%s_d%d_c%d' %(opti, n_dense, n_conv)
            
            model = model_setup(n_dense, n_conv)
            

            # Inicializacija modela in nastavitev:
            #     - kriterijske funkcije,
            #     - optimizatorja,
            #     - metrike.
            model.compile(loss="categorical_crossentropy",
                          optimizer=opti,
                          metrics=['accuracy'])

            # Učenje modela, 10 epik (št._epik x št._vseh_podatkov / batch_size)
            print("Using optimizer: %s to train a CNN with %d dense layers and %d conv layers" %(opti, n_dense, n_conv))
            model.fit(x_train,
                      y_train, 
                      epochs=2, 
                      batch_size=100, 
                      verbose=1,
                      callbacks=[TensorBoard(log_dir='log/%s/' %hparam, histogram_freq=1, write_graph=True)] 
            )
print('Finished!')

Using optimizer: Adam to train a CNN with 2 dense layers and 2 conv layers
INFO:tensorflow:Summary name conv2d_1/kernel:0 is illegal; using conv2d_1/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_1/bias:0 is illegal; using conv2d_1/bias_0 instead.
INFO:tensorflow:Summary name conv2d_2/kernel:0 is illegal; using conv2d_2/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_2/bias:0 is illegal; using conv2d_2/bias_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.
INFO:tensorflow:Summary name dense_2/bias:0 is illegal; using dense_2/bias_0 instead.
INFO:tensorflow:Summary name dense_3/kernel:0 is illegal; using dense_3/kernel_0 instead.
INFO:tensorflow:Summary name dense_3/bias:0 is illegal; using dense_3/bias_0 instead.
Epoch 1/2
60000/60000 [==============